<a href="https://colab.research.google.com/github/nourfayed/inception_transfer_learning/blob/master/project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
cd gdrive/My Drive/Neural Networks/Files


/content/gdrive/My Drive/Neural Networks/Files


In [0]:
!ls


In [0]:
import numpy as np

X_Train=[]
Y_TrainLabels=[]
Testing_images = []

Y_TrainLabels =np.load('labels.npy',allow_pickle=True)
X_Train = np.load('train.npy',allow_pickle=True)
Testing_images = np.load('TestingImages.npy',allow_pickle=True)

print(X_Train.shape)
print("khalas")

#print(Y_TrainLabels)

(5827, 299, 299, 3)
khalas


In [0]:
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
import inception_resnet_v2 as incep_v2
import os
import numpy as np
from tqdm import tqdm
import matplotlib.patches as mpatches
from random import shuffle
from google.colab import files
from PIL import Image
import glob

Features_vector = []
n_rows = 299
n_cols = 299




def define_model(model, is_training):
    model.Image = tf.placeholder(tf.float32, shape=[None, n_rows, n_cols, 3])
    with incep_v2.slim.arg_scope(incep_v2.inception_resnet_v2_arg_scope()):
        model.logits, model.end_points = incep_v2.inception_resnet_v2(model.Image, is_training=False)


sess = tf.Session()

class Model_Class:
    def __init__(self, is_training):
        define_model(self, is_training=is_training)


print("2")

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
with tf.device('/gpu:0'):
    model = Model_Class(False)

print("done")

model.saver = tf.train.Saver()
model.saver.restore(sess,"\inception_resnet_v2_2016_08_30.ckpt")
predictions_model = model.end_points["PreLogitsFlatten"]

print("4")

Labels=[]
for i in range (len(X_Train)):
    img=X_Train[i]
    label=Y_TrainLabels[i]
    tmp = sess.run(predictions_model, feed_dict={model.Image: [img]})
    Features_vector.append(tmp)
    Labels.append(label)



print("5")


Testing_features=[]
for i in range (len(Testing_images)):
  img=Testing_images[i]
  tmp = sess.run(predictions_model, feed_dict={model.Image: [img]})
  Testing_features.append(tmp)
  




print(np.shape(Features_vector))
print(np.shape(Testing_features))





For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

2
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.flatten instead.
done
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from \inception_resnet_v2_2016_08_30.ckpt
4
5
(5827, 1, 1536)
(771, 1, 1536)


In [0]:
Features_vector = np.array(Features_vector)
Features_vector = Features_vector.reshape((5827,1536))
Testing_features=np.array(Testing_features)
Testing_features=Testing_features.reshape((771,1536))

In [0]:

import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression
import numpy as np

#x, y, z = np.shape(Features_vector)
#Features_vector = np.reshape(Features_vector, (x, y * z))
#x,feature_length = np.shape(Features_vector)

input_layer = input_data(shape=[None ,1536], name='input')
#conv1 = conv_2d(input_layer, 32, 5, activation='relu')
#pool1 = max_pool_2d(conv1, 5)
fully_layer = fully_connected(input_layer, 10, activation='relu')
#fully_layer = dropout(fully_layer, 0.5)

#fully_layer2 = fully_connected(fully_layer, 512, activation='relu')
#fully_layer2 = dropout(fully_layer2, 0.6)
print("1")
cnn_layers = fully_connected(fully_layer, 10, activation='softmax')
regression = regression(cnn_layers, optimizer='sgd', learning_rate=0.001, loss='categorical_crossentropy',
                        name='targets')
cnn_model = tflearn.DNN(regression, tensorboard_dir='log', tensorboard_verbose=0)
print("2")
 
  

cnn_model.fit({'input': Features_vector}, {'targets': Labels}, n_epoch=50,
          validation_set=0.1,batch_size=8,  shuffle=True,    
          show_metric=True, run_id='inception+cnn13')
print("khalas l fit")
cnn_model.save('the_cnn_model15.tfl')







Training Step: 32799  | total loss: 2.23261 | time: 34.125s
| SGD | epoch: 050 | loss: 2.23261 - acc: 0.1465 -- iter: 5240/5244
Training Step: 32800  | total loss: 2.22282 | time: 35.192s
| SGD | epoch: 050 | loss: 2.22282 - acc: 0.1443 | val_loss: 2.21723 - val_acc: 0.1252 -- iter: 5244/5244
--
khalas l fit
INFO:tensorflow:/content/gdrive/My Drive/Neural Networks/Files/the_cnn_model15.tfl is not in all_model_checkpoint_paths. Manually adding it.


In [0]:
!pwd

/content/gdrive/My Drive/Neural Networks/Files


In [0]:
#cnn_model = tflearn.DNN(regression, tensorboard_dir='log', tensorboard_verbose=0)
#cnn_model.load('the_cnn_model13.tfl')
predictions=[]

#for i in range (len(Testing_features)):
 
prediction = cnn_model.predict(Testing_features)
print(np.shape(prediction)) 
print(prediction)
for i in range (len(prediction)):
  tmp=-1
  mx=0
  for j in range (len(prediction[i])):
    if(prediction[i][j]>tmp):
      tmp=prediction[i][j]
      mx=j
  predictions.append(mx+1)
 


  

print(np.shape(predictions))
print(predictions)


(771, 10)
[[0.11622455 0.19260702 0.07247151 ... 0.01205625 0.03022451 0.03143079]
 [0.00683985 0.00700419 0.00882042 ... 0.07697919 0.27826744 0.06826711]
 [0.04224209 0.10200424 0.05137166 ... 0.08869446 0.04990113 0.16786417]
 ...
 [0.01694723 0.04470484 0.2729958  ... 0.02880161 0.16552486 0.08142993]
 [0.3466117  0.19153307 0.03024846 ... 0.02940763 0.17501181 0.08592521]
 [0.31138927 0.17548293 0.0065417  ... 0.09205704 0.05036405 0.14861195]]
(771,)
[7, 4, 4, 10, 4, 2, 8, 4, 10, 7, 5, 1, 4, 3, 8, 3, 3, 6, 7, 8, 7, 3, 5, 7, 2, 4, 2, 3, 1, 1, 7, 6, 9, 7, 10, 2, 1, 10, 2, 9, 7, 1, 7, 6, 8, 10, 7, 2, 1, 3, 5, 9, 7, 3, 7, 1, 2, 7, 10, 6, 10, 1, 1, 10, 1, 8, 8, 3, 8, 9, 10, 3, 10, 1, 3, 3, 2, 3, 4, 3, 2, 8, 7, 4, 3, 8, 7, 3, 10, 8, 1, 4, 8, 2, 2, 7, 10, 7, 1, 5, 8, 10, 6, 7, 10, 1, 3, 1, 3, 1, 4, 7, 5, 2, 2, 7, 4, 1, 6, 5, 3, 7, 8, 1, 7, 7, 8, 5, 1, 9, 5, 2, 1, 8, 2, 1, 7, 2, 1, 7, 7, 8, 7, 1, 1, 3, 8, 1, 2, 1, 7, 3, 1, 9, 8, 3, 7, 4, 4, 2, 2, 5, 7, 7, 7, 5, 1, 2, 2, 10, 10, 8, 7, 8, 

In [0]:
import csv

path = '/content/gdrive/My Drive/Neural Networks/test/*.jpg'
pathes = glob.glob(path)

print(len(predictions))
print(len(pathes))

filename = "Model15.csv" 
with open(filename, 'w') as csvfile: 
     
    csvwriter = csv.writer(csvfile) 
 
    for i in range(len(predictions)):
       s=pathes[i][46:len(pathes[i])]
       tmp=[s,predictions[i]]
       csvwriter.writerow(tmp)

    csvfile.close()
    
    
    

771
771


In [0]:
import csv

filename = "testcsv3.csv"
  
# writing to csv file 
with open(filename, 'w') as csvfile: 
    # creating a csv writer object 
    csvwriter = csv.writer(csvfile) 
    tmp=[1,2]
    for i in range(20):
       csvwriter.writerow(tmp)

    csvfile.close()

